# Configure visualization with bitmask segmentations

In [91]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeTiffWrapper,
    MultiImageWrapper,
)
from os.path import join

In [92]:
vc = VitessceConfig(schema_version="1.0.15", name='10x genomics', description='Visium')

In [93]:
dataset = vc.add_dataset(name='Human lymph node').add_object(AnnDataWrapper(
    adata_path=join("processed_data", "alternate.adata.zarr"),
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_locations_path="obsm/spatial",
    obs_set_paths=["obs/clusters"],
    obs_set_names=["Leiden Cluster"],
    obs_feature_matrix_path="obsm/X_hvg",
    feature_filter_path="var/highly_variable",
)).add_object(MultiImageWrapper([
    OmeTiffWrapper(
        img_path=join("processed_data", "alternate.bitmask.ome.tiff"),
        name="Spot segmentations",
        is_bitmask=True
    ),
    OmeTiffWrapper(
        img_path=join("processed_data", "alternate.image.ome.tiff"),
        name="H&E Image",
        is_bitmask=False
    ),
], use_physical_size_scaling=True))

In [94]:
spatial_colored_by_cluster = vc.add_view(cm.SPATIAL, dataset=dataset)
spatial_colored_by_expression = vc.add_view(cm.SPATIAL, dataset=dataset)

layer_controller = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=True)
spot_set_manager = vc.add_view(cm.OBS_SETS, dataset=dataset)
gene_list = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset).set_props(transpose=True)

In [95]:
vc.layout(
    (spatial_colored_by_cluster | spatial_colored_by_expression)
    / ((layer_controller | spot_set_manager) | (gene_list | heatmap))
);

In [96]:
spatial_views = [
    spatial_colored_by_cluster,
    spatial_colored_by_expression,
    layer_controller,
]
all_views = [
    *spatial_views,
    spot_set_manager,
    gene_list,
    heatmap
]

spatial_segmentation_layer_value = [{
    "type": "bitmask",
    "visible": True,
    "index": 0,
    "colormap": None,
    "transparentColor": None,
    "opacity": 1,
    "domainType": "Min/Max",
    "channels": [
        {
          "selection": { "c": 0, "t": 0, "z": 0 },
          "color": [255, 0, 0],
          "visible": True,
          "slider": [0, 1]
        }
    ]
}]
spatial_image_layer_value = [{
    "type": "raster",
    "index": 0,
    "colormap": None,
    "transparentColor": None,
    "opacity": 1,
    "domainType": "Min/Max",
    "channels": [
        {
          "selection": { "c": 0, "t": 0, "z": 0 },
          "color": [255, 0, 0],
          "visible": True,
          "slider": [0, 255]
        },
        {
          "selection": { "c": 1, "t": 0, "z": 0 },
          "color": [0, 255, 0],
          "visible": True,
          "slider": [0, 255]
        },
        {
          "selection": { "c": 2, "t": 0, "z": 0 },
          "color": [0, 0, 255],
          "visible": True,
          "slider": [0, 255]
        }
    ]
}]
vc.link_views(
    spatial_views,
    [ct.SPATIAL_IMAGE_LAYER, ct.SPATIAL_SEGMENTATION_LAYER],
    [spatial_image_layer_value, spatial_segmentation_layer_value]
)
vc.link_views(
    spatial_views,
    [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y],
    [-2.6, 1000, 1000]
)
vc.link_views(
    [spatial_colored_by_cluster, heatmap, spot_set_manager],
    [ct.OBS_COLOR_ENCODING],
    ["cellSetSelection"]
)
vc.link_views(
    [spatial_colored_by_expression, gene_list],
    [ct.OBS_COLOR_ENCODING, ct.FEATURE_SELECTION],
    ["cellSetSelection", ["CR2"]]
)

In [97]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': '10x genomics', 'description': 'Visium', 'datasets': [{'ui…